In [ ]:
import numpy as np
import pandas as pd
from math import log
from scipy.io import arff
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
from sklearn.metrics import auc  
from sklearn.metrics import precision_recall_fscore_support
from random import randrange
from scipy.stats import chi2_contingency
from sklearn.model_selection import KFold

In [ ]:
def calcShannonEnt(dataSet):#计算给定数据集的经验熵(香农熵)
    numEntires = len(dataSet)                     
    labelCounts = {}                                 #保存每个标签(Label)出现次数的字典
    for featVec in dataSet:                        
        currentLabel = featVec[-1]                   
        if currentLabel not in labelCounts.keys():  
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1              
    shannonEnt = 0.0                                 #经验熵(香农熵)
    for key in labelCounts:                       
        prob = float(labelCounts[key]) / numEntires  #选择该标签(Label)的概率
        shannonEnt -= prob * log(prob, 2)            #利用公式计算
    return shannonEnt                                #返回经验熵(香农熵)

def splitDataSet(dataSet, axis, value):#函数说明:按照给定特征划分数据集
    retDataSet = []                                 
    for featVec in dataSet:                          
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]             #去掉axis特征
            reducedFeatVec.extend(featVec[axis+1:])     #将符合条件的添加到返回的数据集
            retDataSet.append(reducedFeatVec)
    return retDataSet                                  

def chooseBestFeatureToSplit(dataSet):#函数说明:选择最优特征
    numFeatures = len(dataSet[0]) - 1                     #特征数量
    baseEntropy = calcShannonEnt(dataSet)                 #计算数据集的香农熵
    lis = []
    for i in range(numFeatures):                        
        #获取dataSet的第i个所有特征
        featList = [example[i] for example in dataSet]
        featList = pd.cut(featList, 20).codes
        uniqueVals = set(featList)                         #创建set集合{},元素不可重复
        newEntropy = 0.0                                   #经验条件熵
        for value in uniqueVals:                          
            subDataSet = splitDataSet(dataSet, i, value)           #subDataSet划分后的子集
            prob = len(subDataSet) / float(len(dataSet))        
            newEntropy += prob * calcShannonEnt(subDataSet)        #根据公式计算经验条件熵
        infoGain = baseEntropy - newEntropy                        #信息增益
        lis.append(infoGain)
    return lis

In [ ]:
def relief(features, labels,times):   
    (n_samples, n_features) = np.shape(features)
    delta = []
    delta_features = []
    delta_index = []
    sample_distance = sap_distance(features)  # 计算每两个样本之间的距离
    new_features = normalize(features)        # 对特征值归一化
 
    '''# 下面开始计算相关统计量，并对各个特征的相关统计量进行比较，最后返回各个特征值相关统计量从高到低的排名
    # 这是将随机选取的样本代入计算出来的delta
    for i in range(0, times):
        randnum = randrange(0, n_samples, 1)        # 生成一个随机数
        one_sample = features[randnum]        # 随机选择一个样本
        (nearhit, nearmiss, nearhit_index, nearmiss_index) = search_near(sample_distance, labels, randnum, features)  # 找出猜中近邻和猜错近邻,nearhit为猜中近邻样本的行向量
        delta.append(relevant_feature(nearhit_index, nearmiss_index, new_features, randnum))  # 计算相关统计量矩阵
    delta = np.asarray(delta)
    for j in range(0, n_features):
        delta_features.append(np.sum(delta[:, j]))
    midd = list(set(delta_features))
    midd.sort(reverse=True)
    for p in midd:
        for q in range(0, len(delta_features)):
            if delta_features[q] == p:
                delta_index.append(q)
    return delta_index'''
    # 这是将所有样本都带入计算的delta
    for i in range(0, n_samples):
        (nearhit, nearmiss, nearhit_index, nearmiss_index) = search_near(sample_distance, labels, i,
                                                                         features)  # 找出猜中近邻和猜错近邻,nearhit为猜中近邻样本的行向量
        delta.append(relevant_feature(nearhit_index, nearmiss_index, new_features, i))  # 计算相关统计量矩阵
    delta = np.asarray(delta)
    for j in range(0, n_features):
        delta_features.append(np.sum(delta[:, j]))
    midd = list(set(delta_features))
    midd.sort(reverse=True)
    for p in midd:
        for q in range(0, len(delta_features)):
            if delta_features[q] == p:
                delta_index.append(q)
    return delta_index


def normalize(features):
    (n_samples, n_features) = np.shape(features)
    fe_max = []
    fe_min = []
    n_deno = []
    new_features = np.zeros((n_samples, n_features))
    for i in range(0, n_features):
        max_index = np.argmax(features[:, i])
        min_index = np.argmin(features[:, i])
        fe_max.append(features[max_index, i])  # 计算每一个特征的最大值
        fe_min.append(features[min_index, i])  # 计算每一个特征的最小值
    n_deno = np.asarray(fe_max) - np.asarray(fe_min)  # 求出归一化的分母
    for j in range(0, n_features):
        for k in range(0, n_samples):
            new_features[k, j] = (features[k, j]-fe_min[j]) / n_deno[j]  # 归一化
    return new_features

def sap_distance(features):
    (n_samples, n_features) = np.shape(features)
    distance = np.zeros((n_samples, n_samples))
    for i in range(0, n_samples):
        for j in range(0, n_samples):
            diff_distance = features[i]-features[j]
            if i == j:
                distance[i, j] = 9999
            else:
                distance[i, j] = euclid_distance(diff_distance)  # 使用欧几里德距离定义样本之间的距离
    return distance

def euclid_distance(diff_distance):
    counter = np.power(diff_distance, 2)
    counter = np.sum(counter)
    counter = np.sqrt(counter)
    return counter

def search_near(sample_distance, labels, randnum, feartures):
    (n_samples, n_features) = np.shape(feartures)
    nearhit_list = []
    nearmiss_list = []
    hit_index = []
    miss_index = []
    for i in range(0, n_samples):
        if labels[i] == labels[randnum]:
            nearhit_list.append(sample_distance[i, randnum])  # 将距离放在一个列表里面
            hit_index.append(i)                                 # 将样本标号放在另一个列表里面
        else:
            nearmiss_list.append(sample_distance[i, randnum])
            miss_index.append(i)
    nearhit_dis_index = nearhit_list.index(min(nearhit_list))   # 算出猜中近邻
    nearhit_index = hit_index[nearhit_dis_index]                # 将猜中近邻的样本标号赋给nearhit_index

    nearmiss_dis_index = nearmiss_list.index(min(nearmiss_list))
    nearmiss_index = miss_index[nearmiss_dis_index]


    nearhit = feartures[nearhit_index]
    nearmiss = feartures[nearmiss_index]

    return nearhit, nearmiss, nearhit_index, nearmiss_index

def relevant_feature(nearhit_index, nearmiss_index, new_features, randnum):
    diff_hit = abs(new_features[nearhit_index]-new_features[randnum])
    diff_miss = abs(new_features[nearmiss_index]-new_features[randnum])
    delta = -np.power(diff_hit, 2)+np.power(diff_miss, 2)
    return delta

In [ ]:
def calc_dex(model, X, y):   #十折交叉验证，返回f1和aoc
    KF = KFold(n_splits=10,random_state=100)
    ans_f1 = 0
    ans_auc = 0
    for train_index,test_index in KF.split(X):
        train_X = X[train_index]
        test_X= X[test_index]
        train_y = np.array(y)[train_index]
        test_y = np.array(y)[test_index]
        model.fit(train_X, train_y)
        predicted = model.predict(test_X)
        pre, rec, f1, sup = precision_recall_fscore_support(test_y, predicted)
        ans_f1+=f1
        try:    #要报error，必须使用try
            ans_auc+=metrics.roc_auc_score(test_y, predicted)  
        except ValueError:
            pass
    ans_f1/=10
    ans_auc/=10
    return ans_f1[0], ans_auc

In [ ]:
def out_put(text, cnt, ans, y):
    if cnt==1:
        print(text)
    clf = DecisionTreeClassifier()
    f1, auc = calc_dex(clf, ans, y)
    print('第{}个数据集使用决策树得到的F1分数是：{}'.format(cnt, f1))
    print('auc分数是：', auc)
    bayers = GaussianNB()
    f1, auc = calc_dex(bayers, ans, y)
    print('使用基于高斯分布的朴素贝叶斯得到的F1分数是：{}'.format(f1))
    print('auc分数是：', auc)
    knn = KNeighborsClassifier()
    f1, auc = calc_dex(knn, ans, y) 
    print('使用KNN得到的F1分数是：{}'.format(f1))
    print('auc分数是：', auc)
    rf = RandomForestClassifier()
    f1, auc = calc_dex(rf, ans, y) 
    print('使用随机森林得到的F1分数是：{}'.format(f1))
    print('auc分数是：', auc)

In [ ]:
lis_adr = ['data/data63886/MW1.arff','data/data63886/CM1.arff','data/data63886/KC1.arff',
'data/data63886/MC1.arff','data/data63886/MC2.arff']
cnt = 0
############全特征##############
for adr in lis_adr:
    cnt+=1
    data = arff.loadarff(adr)    #装载每一个数据
    df = pd.DataFrame(data[0])   
    data = df.T.values.tolist()
    y = data[-1]
    for i in range(len(y)):
        if y[i]==b'Y':
            y[i] = int(1)
        else:
            y[i] = int(0)
    n = int(log(len(data),2)+1)
    data = df.T.values.tolist()[:-1]
    ans = np.array(data)
    ans = ans.T
    out_put('以下是全特征训练运行的结果', cnt, ans, y)

以下是全特征训练运行的结果
第1个数据集使用决策树得到的F1分数是：0.9007844088132181
auc分数是： 0.5372541552650248
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.8902689046633145
auc分数是： 0.6497469088882133
使用KNN得到的F1分数是：0.9378009371844792
auc分数是： 0.4479166666666667
使用随机森林得到的F1分数是：0.9330910405408321
auc分数是： 0.46257608695652175


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no pr

第2个数据集使用决策树得到的F1分数是：0.9011073550878631
auc分数是： 0.4971137543398389
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9000555573321259
auc分数是： 0.5012763323532232
使用KNN得到的F1分数是：0.9185349217866069
auc分数是： 0.3903192204301075
使用随机森林得到的F1分数是：0.9279789205317694
auc分数是： 0.42223114066134093
第3个数据集使用决策树得到的F1分数是：0.8577798023796556
auc分数是： 0.5787618272345892
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.8946584686764801
auc分数是： 0.6449199439766231
使用KNN得到的F1分数是：0.8942182365648957
auc分数是： 0.5717007245484473


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


使用随机森林得到的F1分数是：0.903540014956909
auc分数是： 0.5733379286520754


...

第4个数据集使用决策树得到的F1分数是：0.9904830193819478
auc分数是： 0.4868224421478957
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9800182902362369
auc分数是： 0.520933055543032


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being

使用KNN得到的F1分数是：0.9960958312595078
auc分数是： 0.3998368972553134


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


使用随机森林得到的F1分数是：0.9957683936419424
auc分数是： 0.4161228273890513
第5个数据集使用决策树得到的F1分数是：0.6518266537384184
auc分数是： 0.5769390331890332
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.7726837233749335
auc分数是： 0.6370238095238097
使用KNN得到的F1分数是：0.749175757611617
auc分数是： 0.6137067099567098
使用随机森林得到的F1分数是：0.7580004242179501
auc分数是： 0.7193253968253968


In [41]:
cnt = 0
#############信息增益选择特征###############
for adr in lis_adr:
    cnt+=1
    data = arff.loadarff(adr)
    df = pd.DataFrame(data[0])   
    data = df.T.values.tolist()
    y = data[-1]
    for i in range(len(y)):
        if y[i]==b'Y':
            y[i] = int(1)
        else:
            y[i] = int(0)
    n = int(log(len(data),2)+1)
    data = df.values.tolist()
    bestfeature=chooseBestFeatureToSplit(data)
    m = sorted(bestfeature, reverse = True)
    lis_shan = []    #信息增益选出的特征
    for i in range(n):
        for j in range(len(bestfeature)):
            if m[i]==bestfeature[j]:
                lis_shan.append(j)
                bestfeature[j] = -1
                break
    data = df.T.values.tolist()
    ans = []
    for i in lis_shan:
        ans.append(data[i])
    ans = np.array(ans)
    ans = ans.T
    out_put('以下是信息增益训练运行的结果', cnt, ans, y)

以下是信息增益训练运行的结果
第1个数据集使用决策树得到的F1分数是：0.9110410380333732
auc分数是： 0.4897269011182055
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9083555721394323
auc分数是： 0.4612327877436573
使用KNN得到的F1分数是：0.9418009371844793
auc分数是： 0.4479166666666667
使用随机森林得到的F1分数是：0.9268397750708675
auc分数是： 0.45013028276071754


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no pr

第2个数据集使用决策树得到的F1分数是：0.856263549782445
auc分数是： 0.4066899775002528
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.908156234690734
auc分数是： 0.46959114925674994
使用KNN得到的F1分数是：0.9185349217866069
auc分数是： 0.3903192204301075
使用随机森林得到的F1分数是：0.9082292866583599
auc分数是： 0.38563957848788216
第3个数据集使用决策树得到的F1分数是：0.8795390317713592
auc分数是： 0.6001633383953868
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9095522017832763
auc分数是： 0.6120800536004924
使用KNN得到的F1分数是：0.8942004752290993
auc分数是： 0.5756445274223372


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


使用随机森林得到的F1分数是：0.8964694560536177
auc分数是： 0.5775467861918513


...

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being se

使用KNN得到的F1分数是：0.9960958312595078
auc分数是： 0.3998368972553134


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


使用随机森林得到的F1分数是：0.9953299374576551
auc分数是： 0.3990735880351123
第5个数据集使用决策树得到的F1分数是：0.678330712115542
auc分数是： 0.6207720057720058
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.7652709571130625
auc分数是： 0.6292045454545455
使用KNN得到的F1分数是：0.749175757611617
auc分数是： 0.6137067099567098
使用随机森林得到的F1分数是：0.7070400639279357
auc分数是： 0.6120995670995669


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [42]:
cnt = 0
############卡方选择特征##############
for adr in lis_adr:
    cnt+=1
    data = arff.loadarff(adr)    #装载每一个数据
    df = pd.DataFrame(data[0])   
    data = df.T.values.tolist()
    y = data[-1]
    for i in range(len(y)):
        if y[i]==b'Y':
            y[i] = int(1)
        else:
            y[i] = int(0)
    n = int(log(len(data),2)+1)
    data = df.values.T.tolist()
    ans = []
    for j in range(len(data)-1):
        l = []
        l.append(data[j])
        l.append(y)
        try:    #要报error，必须使用try
            chi2,pval,dof,expected = chi2_contingency(l)
            ans.append(pval) 
        except ValueError:
            pass
    m = sorted(ans, reverse = True)
    lis_K = []
    for i in range(n):
        for j in range(len(ans)):
            if m[i]==ans[j]:
                lis_K.append(j)
                ans[j] = -1
                break
    data = df.T.values.tolist()
    ans = []
    for i in lis_K:
        ans.append(data[i])
    ans = np.array(ans)
    ans = ans.T
    out_put('以下是卡方运行的结果', cnt, ans, y)

以下是卡方运行的结果
第1个数据集使用决策树得到的F1分数是：0.8961554051326539
auc分数是： 0.44816357555487996
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9132161793150528
auc分数是： 0.43942723218810176
使用KNN得到的F1分数是：0.9377191268115638
auc分数是： 0.4439102564102564
使用随机森林得到的F1分数是：0.9282424390378082
auc分数是： 0.44292859193946155
第2个数据集使用决策树得到的F1分数是：0.8657004685912536
auc分数是： 0.39501609208366717


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labe

使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9269092630006808
auc分数是： 0.4
使用KNN得到的F1分数是：0.9185349217866069
auc分数是： 0.3903192204301075
使用随机森林得到的F1分数是：0.9202728500769546
auc分数是： 0.39050124085684434
第3个数据集使用决策树得到的F1分数是：0.9056694078760996
auc分数是： 0.5863807553082536
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9022806814626894
auc分数是： 0.6229181786307534
使用KNN得到的F1分数是：0.9048052712461689
auc分数是： 0.5812940240016663


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


使用随机森林得到的F1分数是：0.9025802523396644
auc分数是： 0.5810625915547745


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'reca

第4个数据集使用决策树得到的F1分数是：0.9893928218167988
auc分数是： 0.40700637736045264
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9763654905897748
auc分数是： 0.5572114670973999


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being

使用KNN得到的F1分数是：0.995770921299567
auc分数是： 0.3996745023721213
使用随机森林得到的F1分数是：0.9952223770143274
auc分数是： 0.39896756641178677
第5个数据集使用决策树得到的F1分数是：0.6536407761485161
auc分数是： 0.5955916305916307
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.7403517075646568
auc分数是： 0.583867243867244
使用KNN得到的F1分数是：0.7261163492510242
auc分数是： 0.6547799422799423
使用随机森林得到的F1分数是：0.6849136321195145
auc分数是： 0.5566071428571429


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being

In [43]:
cnt = 0
############relief提取特征##############
for adr in lis_adr:
    cnt+=1
    data = arff.loadarff(adr)    #装载每一个数据
    df = pd.DataFrame(data[0])   
    data = df.T.values.tolist()
    y = data[-1]
    for i in range(len(y)):
        if y[i]==b'Y':
            y[i] = int(1)
        else:
            y[i] = int(0)
    y = np.array(y)
    n = int(log(len(data),2)+1)
    data = df.values.T
    labels = np.array(y).T
    features = data[:-1].T
    features_importance = relief(features, labels,3)
    lis_rel = []
    for i in range(n):
        lis_rel.append(features_importance[i])
    data = df.T.values.tolist()
    ans = []
    for i in lis_rel:
        ans.append(data[i])
    ans = np.array(ans)
    ans = ans.T
    out_put('以下是relief训练运行的结果', cnt, ans, y)

以下是relief训练运行的结果
第1个数据集使用决策树得到的F1分数是：0.9170120309715456
auc分数是： 0.6033423448532143
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.8535156942717202
auc分数是： 0.5247827691632039
使用KNN得到的F1分数是：0.9313611990250644
auc分数是： 0.43782051282051276
使用随机森林得到的F1分数是：0.9473433541512659
auc分数是： 0.5475


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no pr

第2个数据集使用决策树得到的F1分数是：0.8983902378228471
auc分数是： 0.44071391240850366
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9098999123878017
auc分数是： 0.46930160194829273
使用KNN得到的F1分数是：0.9230058695784502
auc分数是： 0.40742358934169276
使用随机森林得到的F1分数是：0.931235218968428
auc分数是： 0.425
第3个数据集使用决策树得到的F1分数是：0.8672513067480938
auc分数是： 0.5915584823593023
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.8828739723659534
auc分数是： 0.6565348817483468
使用KNN得到的F1分数是：0.891795336362742
auc分数是： 0.5717541927340121


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


使用随机森林得到的F1分数是：0.8991539137419003
auc分数是： 0.5607458919742967


...

第4个数据集使用决策树得到的F1分数是：0.9924149064210714
auc分数是： 0.40019174445296396
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.9322895871275376
auc分数是： 0.5419673577318308


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being

使用KNN得到的F1分数是：0.996040751723028
auc分数是： 0.3997281425935563


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


使用随机森林得到的F1分数是：0.9957122410709076
auc分数是： 0.3994017596901698
第5个数据集使用决策树得到的F1分数是：0.7215933400291995
auc分数是： 0.5970183982683983
使用基于高斯分布的朴素贝叶斯得到的F1分数是：0.6647735597735598
auc分数是： 0.6074188311688313
使用KNN得到的F1分数是：0.6482940004611831
auc分数是： 0.4858658008658009
使用随机森林得到的F1分数是：0.7655686659401829
auc分数是： 0.7107683982683983


In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 